In [3]:
import numpy as np
import matplotlib.pyplot as plt
import rawpy
from tqdm.auto import tqdm
import cv2

In [4]:
def process_raw(path_raw_image, path_to_save_image):
    
    #Assuming path_raw_image, path_to_save_image is a string
    
    raw = rawpy.imread(path_raw_image)
    array = np.array(raw.raw_image_visible)
    height, width = array.shape
    demosaiced_img = np.zeros((height, width, 3), dtype=np.uint16)

    
    for row in tqdm(range(1, height - 1)):
        for col in range(1, width - 1):
            if (row % 2 == 1) and (col % 2 == 1):  # Blue pixel
                demosaiced_img[row, col, 0] = array[row, col]  # Blue
                demosaiced_img[row, col, 1] = (array[row, col - 1] + array[row, col + 1] +
                                               array[row - 1, col] + array[row + 1, col]) // 4  # Green
                demosaiced_img[row, col, 2] = (array[row - 1, col - 1] + array[row - 1, col + 1] +
                                               array[row + 1, col - 1] + array[row + 1, col + 1]) // 4  # Red
            elif (row % 2 == 0) and (col % 2 == 0):  # Red pixel
                demosaiced_img[row, col, 0] = (array[row - 1, col - 1] + array[row - 1, col + 1] +
                                               array[row + 1, col - 1] + array[row + 1, col + 1]) // 4  # Blue
                demosaiced_img[row, col, 1] = (array[row, col - 1] + array[row, col + 1] +
                                               array[row - 1, col] + array[row + 1, col]) // 4  # Green
                demosaiced_img[row, col, 2] = array[row, col]  # Red
            else:  # Green pixel
                demosaiced_img[row, col, 1] = array[row, col]  # Green
                if row % 2 == 0:
                    demosaiced_img[row, col, 0] = (array[row - 1, col] + array[row + 1, col]) // 2  # Blue
                    demosaiced_img[row, col, 2] = (array[row, col - 1] + array[row, col + 1]) // 2  # Red
                     
                else:
                    demosaiced_img[row, col, 0] = (array[row, col - 1] + array[row, col + 1]) // 2  # Blue
                    demosaiced_img[row, col, 2] = (array[row - 1, col] + array[row + 1, col]) // 2  # Red
                    
            
            
            
    demosaiced_img = demosaiced_img[1:-1, 1:-1]
    print(demosaiced_img.min, demosaiced_img.max)
    
    min_val = np.min(demosaiced_img)
    max_val = np.max(demosaiced_img)
    print("Max value:", max_val, "Min Value:", min_val)
    scaled_image = ((demosaiced_img - min_val) / (max_val - min_val)) * 255
    scaled_image = scaled_image.astype(np.uint8)
    
    gamma = 0.45
    lower_percentile, upper_percentile = np.percentile(scaled_image, [0.01, 99.99])
    normalized_image = (scaled_image - lower_percentile) / (upper_percentile - lower_percentile)
    normalized_image[normalized_image < 0] = 0
    normalized_image[normalized_image > 1] = 1
    gamma_corrected_image = np.power(normalized_image, gamma)
    final_image = (gamma_corrected_image * (upper_percentile - lower_percentile)) + lower_percentile
    final_image = np.clip(final_image, 0, 255).astype(np.uint8)
    
    
    avg_r = np.mean(final_image[:,:,2])
    avg_g = np.mean(final_image[:,:,1])
    avg_b = np.mean(final_image[:,:,0])
    avg_gray = (avg_r + avg_g + avg_b) / 3
    final_image[:,:,2] = np.clip((final_image[:,:,2] / avg_r) * avg_gray, 0, 255)
    final_image[:,:,1] = np.clip((final_image[:,:,1] / avg_g) * avg_gray, 0, 255)
    final_image[:,:,0] = np.clip((final_image[:,:,0] / avg_b) * avg_gray, 0, 255)   
    
    cv2.imwrite(path_to_save_image+".png", final_image)

In [5]:
process_raw("test_image.CR3", "Final_image2")

  0%|          | 0/4658 [00:00<?, ?it/s]

<built-in method min of numpy.ndarray object at 0x000002001C0D18F0> <built-in method max of numpy.ndarray object at 0x000002001C0D18F0>
Max value: 16383 Min Value: 2029
